<h1>Nuclear Assemblies and Detectors</h1>

This Jupyter notebook is meant to act as a library of tools to work with nuclear pin assemblies and fluence detectors to take measurements of joint spectra.

Written by Jorge Paz-Peñuelas Oliván for his Dissertation in Physics at the Faculty of Sciences of the University of Zaragoza.
October and November 2023. Implementation on Dec 2023 onwards.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import copy

    # Turn off plots display
#matplotlib.use("agg") 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


This notebook is written to be run without output as a module implementation for further work using the ```import_ipynb``` module, as below.

```
import import_ipynb
import NuclearAssemblies as NA 
``` 
or 
```
import import_ipynb
from NuclearAssemblies import *
```

Each of the three classes defined here (```spectre```, ```assembly```, ```detector```) will have a ```class_test()``` function to run all the default testing code (this is simply to make sure everything runs smoothly).

These will be run here on code blocks specifically used for testing, which we will denote with ```# -- TESTING BLOCK -- ```, in addition to any other specific tests the user wishes to do within this notebook. Running or ignoring said test code blocks will be toggled by the variable ```TEST```, which will simply take value 1 for running and 0 for ignoring.

This is to avoid the programm running all the testing code everytime the notebook is implemented as module.

In [2]:
TEST=0

<h2>The spectre class</h2>

We begin by defining a class to contain everything to do with spectres of gamma emission of nuclear fuel pins. We could alternatively have defined all these as global functions but this seemed cleaner. This also allows us to store information relative to a particular spectre.

The attributes of the spectre class are:

- The distance to the pin at which the measurement was taken (```self.distance```). At the ORNL IFEL this distance was 502 cm.
- An assigned name to the spectre (```self.name```)
- The spectre in itself (```self.data```). It will be a two dimensional array with lenght 3 at the first dimension. ```self.data[0]``` will contain the energies in MeV, ```self.data[1]``` the fluence in number of photons per square centimeter and ```self.data[2]``` the relative error of these mesurements.

In [3]:
class spectre:
    
    
    def __init__(self,filename=None,folderpath='*Spectra/',measureDistance=5.02):
        '''Initialization of a spectre instance.
        
        __init__(self,filename=None,folderpath='Spectra/',measureDistance=1e-5)
        '''
        
        self.distance=measureDistance
        self.data=self._read(filename,folderpath)
        self.name=filename
    

    def _read(self,filename,folderpath='Spectra/'):
        '''To read the gamma emission spectre from a data file.
    
        read(self,filename,folderpath='Spectra/')
    
        Taylored to accomodate for the files provided by Jaime Ruz Armendáriz. The data has been taken 
        at <institute>, <location>.
        The first two lines and only the first two will be ignored. The first column is the Energy in MeV,
        the second the fluence in photons per squared cm and the third the relative error.
        '''
    
        pathname=folderpath+filename
        file=open(pathname,'r')
    
            # two dataless lines
        file.readline()
        file.readline()
    
        data = np.array([[float(line.split()[j]) for j in range(len(line.split()))] for line in file])
    
        file.close()
        
        return data.T
    

    # -- Plotting methods --

    def draw(self,ax=None,figsize=(10,3),Erange=(None,None),Eunits='keV',yError=0,titlename=None,title=None):
        '''To draw a spectre or return the corresponding axes instance.
    
        draw(self,ax=None,figsize=(10,3),Erange=(None,None),Eunits='keV',yError=0,titlename=None,title=None)
        
        - ax='None' to display the plot and type(ax)=Axes to return the modified axes instance.
        '''
        
        mode='axes'
        if ax==None:
            fig, ax = plt.subplots(figsize=figsize)
            mode='plot'
    
        if Eunits=='keV':
            ax.set_xlabel('Energy (keV)')
            energies=1000*self.data[0]
        elif Eunits=='MeV':
            ax.set_xlabel('Energy (MeV)')
            energies=self.data[0]
        else:
            raise Exception('Wrong selection of energy units.')
            
         # We establish the limiting Energy indices.
        i=0
        if Erange[0]==None: 
            EMinIdx=0
        else:
            while energies[i]<Erange[0] and i<len(energies)-1:
                i+=1
            EMinIdx=i
        if Erange[1]==None: 
            EMaxIdx=len(energies)-1
        else:
            while energies[i]<Erange[1] and i<len(energies)-1:
                i+=1
            EMaxIdx=i
            
         # We proceed with plotting.
        barwidth=energies[1]-energies[0]
        if yError:
            yerr=self.data[1]*self.data[2]
            ax.bar(energies[EMinIdx:EMaxIdx],self.data[1][EMinIdx:EMaxIdx],width=barwidth,log=True,yerr=yerr)
        else:
            ax.bar(energies[EMinIdx:EMaxIdx],self.data[1][EMinIdx:EMaxIdx],width=barwidth,log=True)
        
            # We define the title
        if title==None:
            if titlename==None:
                titlename=self.name
            title='Spectre of '+titlename
            
        ax.set_title(title)
        ax.set_ylabel('Fluence ($\gamma$/cm$^2$)')
        
        if mode=='plot': 
            plt.show()
            if figsize[0]<=6:
                print('WARNING: Detail may be lost in small figure sizes.')
                print('For the most accurate representation, increase figure size as much as possible.')
        else:
            return ax
        
    def compare(self,spectre2,ax=None,figsize=(10,3),Eunits='keV',yError=0,color='red',
                titlename=None,title=None,linthresh=None):
        '''To compare two emission spectres.
        
        compare(self,spectre2,ax=None,figsize=(10,3),Eunits='keV',yError=0,color='red',\
titlename=None,title=None,linthresh=None)
        
        The difference between two spectres is computed as the current instance minus the provided one.
        - Spectre2 must be another instance of the spectre class.
        - If ax='difference' the spectreDif array is returned. If ax=None a symlog bar plot is made of 
        spectreDif. If ax is an Axes class instance it is modified with the plot and returned.
        - The other arguments are used to configure the axes instance.
        '''
        
            # We check that spectre2 is a spectre class instance
        if type(spectre2)!=spectre:
            raise Exception('spectre2 is not a spectre class instance!')
            
            # We check that both spectres were taken at the same distance
        if not self.distance-spectre2.distance<1E-5:
            raise Exception('Measurement distance of spectre2 does not coincide!')
        
            # We check the energy range is the same, and if it isn't we reduce to the smallest one
        lenDif=len(self.data[0])-len(spectre2.data[0])
        if lenDif>0:
            self.data=np.delete(self.data,np.arange(-lenDif,0),axis=1) 
        elif lenDif<0:
            spectre2.data=np.delete(spectre2.data,np.arange(lenDif,0),axis=1)
            
            # We compute the spectre difference
        spectreDif=self.data.copy()
                # We add the variance
        spectreDif[2]=(spectreDif[2]*spectreDif[1])**2+(spectre2.data[0]*spectre2.data[1])**2
                # We compute the difference
        spectreDif[1]=spectreDif[1]-spectre2.data[1]
                # We compute the relative error
        spectreDif[2][1:]=[np.sqrt(spectreDif[2][i])/spectreDif[1][i] for i in range(1,len(spectreDif[2]))]
        
        if ax=='difference':
            return spectreDif
            
            # We plot the comparison
        mode='axes' 
        if ax==None:
            fig, ax = plt.subplots(figsize=figsize)
            mode='plot'
    
        if Eunits=='keV':
            ax.set_xlabel('Energy (keV)')
            energies=1000*spectreDif[0]
        elif Eunits=='MeV':
            ax.set_xlabel('Energy (MeV)')
            energies=spectreDif[0]
        else:
            raise Exception('Wrong selection of energy units.')
        
        barwidth=energies[1]-energies[0]
        if yError:
            yerr=spectreDif[1]*spectreDif[2]
            ax.bar(energies,spectreDif[1],width=barwidth,yerr=yerr,color=color)
        else:
            ax.bar(energies,spectreDif[1],width=barwidth,color=color)
        
        non0difIdx=[idx for idx in range(len(spectreDif[1])) if spectreDif[1][idx]!=0]
        non0dif=spectreDif[1][non0difIdx]
        
            # We generate the symlog scale
        if linthresh==None:
            if len(non0dif)>0:
                linthresh=10.0**int(np.log10(np.min(np.abs(non0dif))))/2
            else:
                linthresh=1
        
        myscale=matplotlib.scale.SymmetricalLogScale(1,linthresh=linthresh)
        ax.set_yscale(myscale)
        
            # We generate the yticks list
        minTickFluenceDif=int(np.log10(np.min(np.abs(non0dif))))
        maxTickFluenceDif=int(np.log10(np.max(np.abs(non0dif))))
        explist=np.arange(minTickFluenceDif,maxTickFluenceDif,2)
        ticklist=np.append(10.0**explist,-1*10.0**np.flip(explist))
        
        ax.set_yticks(ticklist)
        
            # We define the title
        if title==None:
            if titlename==None:
                titlename=self.name+' vs '+spectre2.name
            title=titlename
        
        ax.set_title(title)
        ax.set_ylabel('Fluence difference ($\gamma$/cm$^2$)')
        
        if mode=='plot': 
            plt.show()
            if figsize[0]<=6:
                print('WARNING: Detail may be lost in small figure sizes.')
                print('For the most accurate representation, increase figure size as much as possible.')
        else:
            return ax
        
    def sum_spectre(self,spectre2,factor=1,sumErrors=True,verbose=False):
        '''To sum two spectres of possibly different lenghts'''
        
        newSpect=copy.deepcopy(spectre2)
        lenDif=len(newSpect.data[0])-len(self.data[0])
        if lenDif>0:
            newSpect.data=np.delete(newSpect.data,np.arange(-lenDif,0),axis=1) 
        elif lenDif<0:
            self.data=np.delete(self.data,np.arange(lenDif,0),axis=1)
            if np.any(self.data[0]!=newSpect.data[0]):
                raise Exception('Energies do not coincide. Spectra not summed.')
            if verbose:
                print('WARNING: energy range reduced')
        
            # We do the proceeding operations. 
        self.data[1]=self.data[1]+factor*newSpect.data[1]
        if sumErrors:
            self.data[2]=np.sqrt((self.data[1]*self.data[2])**2+(factor*newSpect.data[1]*newSpect.data[2])**2)
                    # We add the variance
            no0=np.where(self.data[1]!=0)[0]
            self.data[2]=[self.data[2][i]/self.data[1][i] if i in no0 else 0 for i in range(len(self.data[2]))]
                    # We return to relative error

        
        
        
# -- Testing functions --

def spectre_test():
    print('\033[1m_test():\033[0m Spectres can be plotted directly with')
    print('  -->  spectre(\'4JR_atm106_spectrum.txt\').draw()')
    spectre('4JR_atm106_spectrum.txt').draw()

    print('\033[1m_test():\033[0m The plot can be placed within a figure defined on the main code:')
    print('  -->  fig,ax=plt.subplots(figsize=(6,2))')
    print('  -->  ax=spectre(\'4JR_atm106_spectrum.txt\').draw(ax)')
    print('  -->  plt.show()')
    fig,ax=plt.subplots(figsize=(6,2))
    ax=spectre('4JR_atm106_spectrum.txt').draw(ax)
    plt.show() 
    print('\033[1m_test():\033[0m Note that displaying smaller affects the accuracy of representation of \
fluence peaks.')
    
    print('\n\033[1m_test():\033[0m We can also compare two spectres.')
    print('  -->  spectre(\'4JR_atm106_spectrum.txt\').compare(spectre(\'4JR_moxrod_spectrum2.txt\'))')
    spectre('4JR_atm106_spectrum.txt').compare(spectre('4JR_moxrod_spectrum2.txt'))
    
    

To speed up execution, especially when generating datasets for NN training, we are going to store the 3 provided spectra in a dictionary to avoid opening and reading the same file repeatedly (that would be up to $n^2kl$, which can be around 50k times for a trivial dataset).

In [4]:
SpectraDict = {
    '4JR_atm106_spectrum.txt': spectre('4JR_atm106_spectrum.txt'),
    '4JR_moxrod_spectrum2.txt': spectre('4JR_moxrod_spectrum2.txt'),
    '4JR_tmirod_uo2_spectrum.txt': spectre('4JR_tmirod_uo2_spectrum.txt'),
}

In [5]:
            # -- TESTING BLOCK --
# TEST=0
if TEST:
    
    spectre_test()

<h2>The assembly class</h2>

Here we define a class to represent the pin assemblies. Its attributes are:

- The number of pins per side $s$ of the assembly (```self._nPinSide```). We will assume square assemblies with an odd number of pins per side.
- The spacing, or gap, between the pins (```self.gap```). This represents the distance in meters between the centres of two consecutive pins.
- The pinslots attribute (```self.pinslots```) will be a three-dimentional array containing the convex linear combination coefficients of the spectres in the spectra dictionary for each pin in the assembly.
- The spectra attribute (```self.spectra```) is an array of the filenames of the spectra of each pin, which will work in conjunction with ```self.pinslots```.

To explain further, the assembly will be initialised via a file containing all the spectra, which will become the ```self.spectra``` attribute. This file will contain either the spectra file names, which when necessary will be directly read from disk or copied from the dictionary, the word 'convex' for making convex linear combinations (coefficients adding 1) of the dictionary spectra for randomness, or a '-' for a non-present pin. Then the ```self.pinslots``` attribute will be created, assigning to each pin a 3-tuple (or more if the dictionary expands) of convex coordinates. For example, if for a specific pin ```self.spectra``` takes value '4JR_atm106_spectrum.txt', then ```self.pinslots``` will be assigned ```[1,0,0]```. Note that this way, if we sum all values in ```self.pinslots```, we will obtain the total number of present pins $n$.

Let us too dedicate some time to explain the different location systems for the pins. The first two dimentional indexes of the pinslots array will be understood as matricial indexes: that is, the first index will denote the row and the second the column, starting at 0 as array indexing works in python, and will increase downwards and to the right. We will also work with positions. These will be integers centered on the middle pin, and will work similar to (but will not be) cartesian coordinates, which is to say that the first integer will refer to the x coordinate, and increase to the right, and the second will refer to the y index, and increase upwards. Finally, we will work with a pure cartesian coordinate system which will include the exact location (measured in meters) centered on the middle pin, and will also be consistent with cartesian coordinates. We draw the following three matrices to illustrate the three location systems, assuming a square assembly of 3 pins per side with a 0.1m spacing.

\begin{equation}
\underbrace{\begin{matrix}
(0,0) & (0,1) & (0,2)\\
(1,0) & (1,1) & (1,2)\\
(2,0) & (2,1) & (2,2)\\
\end{matrix}}_{\text{Index system}}
\qquad
\underbrace{\begin{matrix}
(-1,1) & (0,1) & (1,1)\\
(-1,0) & (0,0) & (1,0)\\
(-1,-1) & (0,-1) & (1,-1)\\
\end{matrix}}_{\text{Position system}}
\qquad
\underbrace{\begin{matrix}
(-0.1,0.1) & (0.0,0.1) & (0.1,0.1)\\
(-0.1,0.0) & (0.0,0.0) & (0.1,0.0)\\
(-0.1,-0.1) & (0.0,-0.1) & (0.1,-0.1)\\
\end{matrix}}_{\text{Coordinates system}}
\end{equation}

We are going to assume that the assembly size $s$ is odd. Let us work out the change of variables. We denote the location systems by $[i,j]$, $(k,l)$, and $(x,y)$ respectively. It is easy to see that the Location system is an intermediate one, and that the change from Position to Coordinates is trivial (it is enough to rescale by ```assembly.gap```), so we only need to define the change of variable between the two first systems. That is

\begin{equation}
f\left([i,j]\right)=\left(j-\frac{s-1}{2},-i+\frac{s-1}{2}\right)=(k,l)\ ,
\end{equation}

\begin{equation}
g\left((k,l)\right)=\left[-l+\frac{s-1}{2},k-\frac{s-1}{2}\right]=[i,j]\ .
\end{equation}

Further, the assembly class has been written to ne initiated by default as a standard nuclear assembly (as per https://syeilendrapramuditya.wordpress.com/2009/04/14/standard-pwr-nuclear-fuel-assembly-17x17-technical-specification/). That is, a 17 by 17 assembly with 0.8 cm fuel pellets and 1cm fuel rods (pins) and a 1.25 cm ```assembly.gap```. We will not consider the instrumentation and control rod guide thimbles: they will be taken as normal fuel pins. The plotting methods of the class will reflect this.


In [6]:
pelletDiam=0.008
pinDiam=0.01

In [7]:
class assembly:
    '''The assembly class contains all information about a square assembly of nuclear fission fuel pins.
    
    self._nPinSide will be a two-tuple atribute that holds the number of pins per side. The default is 17 by 17.
    self.gap will be the spacing in metres between pins. The default is 0.0125m .
    self.pinslots will be a square matrix of size self._nPinSide containing tuples of size len(SpectraDict) which 
        will represent in which position there are pins in our assembly. self.pinslots will be defined from 
        self.spectra .
    self.spectra will be a square string matrix of size self._nPinSide which will hold the name of the datafile
        that corresponds to the spectre of the pin of that same position. It will be the file given when 
        defining an assembly.
        
    INITIALIZATION: __init__(self,assemblyspectra='null',folderpath='*AssemblySpect/')
    assemblyspectra is a file containing information on the assembly dimentions and individual pin spectres.
    '''
    
    def __init__(self,assemblyspectra='null',folderpath='*AssemblySpect/'):   # We initialize the class instance
        
            # We initialize the size (self._nPinSide), the gap (self.gap), and the spectra of the pins (self.spectra)
        if assemblyspectra=='null':
            self._nPinSide=17   # Default assembly size
            self.gap=0.0125   # Default assembly gap
            self.spectra=[['-' for i in range(self._nPinSide)] for j in range(self._nPinSide)]   # Default spectra
        else:
            assemblyspectra=folderpath+assemblyspectra
            file=open(assemblyspectra,'r')
            line=file.readline()
            while line[0]=='#':
                line=file.readline()
            if line.split()[0]=='AssemblyGap(m):':
                if line.split()[1]=='Standard':
                    self.gap=0.0125
                else:    
                    self.gap=float(line.split()[1])
            else: 
                raise Exception('Assembly gap size not provided.')
                
            self.spectra = [[line.split()[i] for i in range(len(line.split()))] for line in file]
            
            file.close()
            
            if np.shape(self.spectra)[0]!=np.shape(self.spectra)[1]:
                raise Exception('Non-square assembly.')
            else:
                self._nPinSide = np.shape(self.spectra)[0]
                if self._nPinSide%2==0:
                    print('Warning: Assembly pin number per side is even.')
        
            # We initialize the pin slots (self.pinslots) 
        self._asign_pinslots()
                
    # -- Auxiliar functions --

    def _asign_pinslot(self,spectrename):
        pinslot=np.zeros(len(SpectraDict))
        if spectrename in list(SpectraDict.keys()):
            idx=np.where(np.array(list(SpectraDict.keys()))==spectrename)[0][0]
            pinslot[idx]=1
        elif spectrename=='convex':
            pinslot=np.random.dirichlet(np.ones(len(SpectraDict)),size=1)[0]
        return pinslot
    
    def _asign_pinslots(self):
        self.pinslots=np.zeros((*np.shape(self.spectra),len(SpectraDict)))
        for i in range(self._nPinSide):
            for j in range(self._nPinSide):
                self.pinslots[i][j]=self._asign_pinslot(self.spectra[i][j])
    
    # -- Position system conversion methods -- 
    
    def idx2pos(self,IdxPinList):
        '''To change pin indexes to position.
        
        idx2pos(self,IdxPinList)
        
        The variable IdxPinList must be a 1D array (a list might give problems) of two-tuples. Each tuple will 
        represent a pin from the array self.pinslots. The tuples will be read as the indexes of self.pinslots
        '''

            # No checking the indexes. I will assume that if someone inputs indexes they know what they are doing
        
        centre=(self._nPinSide-1)/2
        A=np.array([[int(IdxPinList[m][1]-centre),int(-IdxPinList[m][0]+centre)] for m in range(len(IdxPinList))])
        return A
     
    def pos2idx(self,PosPinList):
        '''To change pin position to indexes.
        
        pos2idx(self,PosPinList)
        
        The variable PosPinList must be be a 1D array (a list might give problems) of two-tuples. Each tuple will 
        represent a pin from the array self.pinslots. The tuples will be interpreted as positions.
        '''

        centre=(self._nPinSide-1)/2
        if np.max(abs(PosPinList))>centre: 
                raise Exception('Positions out of range.')
        A=np.array([[int(-PosPinList[m][1]+centre),int(PosPinList[m][0]+centre)] for m in range(len(PosPinList))])
        return A
     
    # The change between position and coordinates will be as simple 
    # as multiplying (pos2coord) or dividing (coord2pos) by self.gap
                          
    
    # -- Assembly modification methods
    
    def pop_pin(self,PopPinList,by='position'):
        '''To remove pins from self.pinslots.
        
        pop_pin(self,PopPinList,by='position')
        
        The variable PopPinList must be a list of two-tuples. Each tuple will represent a pin to be removed from 
        the array self.pinslots. The second argument specifies how the two-tuples are interpreted: 
        by='index' : the tuples will be read as the indexes of self.pinslots,
        by='position' : the tuples will be read as the indexes-O, where O are the indexes of the centre of the 
            assembly,
        by='coordinates' the tuples will be read as the coordinates of the pins. 
        Both elements of each tuple must make sense, or the method will raise a ValueError.
        '''
        
        if not isinstance(PopPinList,np.ndarray):
                PopPinList=np.array(PopPinList)
        
        if np.any(PopPinList=='All'):
            self.pinslots=np.zeros((*np.shape(self.spectra),len(SpectraDict)))
            self.spectra=[['-' for i in range(self._nPinSide)] for j in range(self._nPinSide)]
            
        else:
                # We make sure the input has the correct shape
            if np.shape(PopPinList)[1]!=2: 
                raise ValueError('Must provide a list of two-tuples')
        
                # We write the two-tuples in the index system
            if by=='coordinates':
                PopPinList=PopPinList/self.gap 
                by='position'
            
                # We check if all entries represent an integer
            e=1E-6 
            if not np.all(PopPinList-np.round(PopPinList)<e*np.ones(np.shape(PopPinList))):   
                raise Exception('Non-integers in list')
            
            if by=='position':  
                    # We check if the tuple is consistent with the position system
                PopPinList=self.pos2idx(PopPinList) 
                by='index'
            
            if by=='index':
                    # We will assume that if someone inputs indexes they know what they are doing
                for i in range(len(PopPinList)):
                    self.pinslots[PopPinList[i][0]][PopPinList[i][1]]=np.zeros(len(SpectraDict))
                    self.spectra[PopPinList[i][0]][PopPinList[i][1]]='-'
            else:
                raise Exception('List is not given in any available location system.')
            
            
    def add_pin(self,AddPinList,SpectraList,by='position'):
        '''To add pins to self.pinslots.
        
        add_pin(self,AddPinList,SpectraList,by='position')
        
        The variable PopPinList must be a list of two-tuples. Each tuple will represent a pin to be removed from 
        the array self.pinslots. The second argument specifies how the two-tuples are interpreted: 
        by='index' : the tuples will be read as the indexes of self.pinslots,
        by='position' : the tuples will be read as the indexes-O, where O are the indexes of the centre of the 
            assembly,
        by='coordinates' the tuples will be read as the coordinates of the pins. 
        Both elements of each tuple must make sense, or the method will raise a ValueError.
        
        The variable SpectralList will be the list of filenames of the spectra corresponding to the added pins.
        It must have the same length as AddPinList.
        
        Alternatively, one can choose AddPinList='All' to make self.pinslots 1 at every index, and will have to 
        provide a two dimensional list of filenames for self.spectra
        '''
        
        if not isinstance(AddPinList,np.ndarray):
                AddPinList=np.array(AddPinList)
        
        if np.any(AddPinList=='All'):
            if np.all(np.shape(SpectraList)!=(self._nPinSide,self._nPinSide)):
                 raise Exception('Missing spectra.')
            self.pinslots=[[self._asign_pinslot(SpectraList[i][j]) for i in range(self._nPinSide)] for j in range(self._nPinSide)]
            self.spectra=SpectraList
            
        else:
                # We make sure all pins are provided with their corresponding spectre.
            if len(AddPinList)!=len(SpectraList):
                raise Exception('Missing spectra.')
        
                # We make sure the input has the correct shape
            if np.shape(AddPinList)[1]!=2: 
                raise ValueError('Must provide a list of two-tuples')
        
                # We write the two-tuples in the index system
            if by=='coordinates':
                AddPinList=AddPinList/self.gap 
                by='position'
        
                # We check if all entries represent an integer
            e=1E-6 
            if not np.all(AddPinList-np.round(AddPinList)<e*np.ones(np.shape(AddPinList))):   
                raise Exception('Non-integers in list')
            
            if by=='position':  
                    # Here we check if the tuple is consistent with the position system
                AddPinList=self.pos2idx(AddPinList)
                by='index'
            
            if by=='index':
                # We will assume that if someone inputs indexes they know what they are doing
                for i in range(len(AddPinList)):
                    self.pinslots[AddPinList[i][0]][AddPinList[i][1]]=self._asign_pinslot(SpectraList[i])
                    self.spectra[AddPinList[i][0]][AddPinList[i][1]]=SpectraList[i]
            else:
                raise Exception('List is not given in any available location system.')
                
    def pop_pinRand(self,nPins=1):
        '''To remove nPins randombly from the assembly. Returns an array of the popped pins.
        '''
        
        if nPins>=np.sum(self.pinslots):
            self.pop_pin(['All'])
            print('All pins removed')
            return self.gen_pincoord()/self.gap
        
        else:
            poppedPins=[[0,0]]
            while nPins>0:
                currentPinpos=self.gen_pincoord()/self.gap
                idxs=[int(num) for num in len(currentPinpos)*np.random.rand(nPins)]
                idxs=np.unique(idxs)  # We remove repeated indices
                nPins-=len(idxs)
                # We pop the pins
                pin2pop=np.round(currentPinpos[idxs]) # np.round() to avoid errors
                self.pop_pin(pin2pop)
                poppedPins=np.append(poppedPins,pin2pop,axis=0)
            return np.delete(poppedPins,0,0)
    
    # -- Generation of pin coordinates for plotting --
        
    def gen_pincoord(self): 
        '''To generate coordinates for the slots where there are pins to plot.
        '''
        
        pincoord=np.zeros((self._nPinSide**2,2))
        delindex=[]
        for j in range(self._nPinSide**2):
            if self.spectra[int(j/self._nPinSide)][j%self._nPinSide]!='-':
                pincoord[j]=self.gap*self.idx2pos([[int(j/self._nPinSide),j%self._nPinSide]])
            else: delindex.append(j)
        pincoord=np.delete(pincoord,delindex,axis=0)
        
        return pincoord
    
    def gen_nopincoord(self):
        '''To generate coordinates for the slots where there are pins to plot.
        '''
        
        nopincoord=np.zeros((self._nPinSide**2,2))
        delindex=[]
        for j in range(self._nPinSide**2):
            if self.spectra[int(j/self._nPinSide)][j%self._nPinSide]=='-':
                nopincoord[j]=self.gap*self.idx2pos([[int(j/self._nPinSide),j%self._nPinSide]])
            else: delindex.append(j)
        nopincoord=np.delete(nopincoord,delindex,axis=0)
        
        return nopincoord
    

    # -- Plotting methods --
    
    def draw(self,ax=None,size=4.82,pinsize='default',DarkCentre=1): 
        '''A drawing of the assembly.
        
        draw(self,ax=None,size=4.82,pinsize='default',DarkCentre=1)
        
        - ax=None to display the plot and ax an Axes instanve to be returned modified.
        - DarkCentre is a variable to control whether we want to colour the central pin in a greyish colour so 
        it is more easily picked out.
        '''
        
        pincoord=self.gen_pincoord().T
        nopincoord=self.gen_nopincoord().T
        
        
        if pinsize=='default':
            psz=((pinDiam/self.gap)*(size/(self._nPinSide-1))*35.4*1.3)**2
                        # 35.4 is a fitted predicted parameter. 1.3 is trial and error
        else: 
            psz=pinsize
        
        mode='axes'
        if ax==None:
            fig,ax=plt.subplots(figsize=(size,size))
            mode='plot'
            
        ax.scatter(pincoord[0],pincoord[1],s=psz,color='steelblue')
        ax.scatter(nopincoord[0],nopincoord[1],s=psz,color='steelblue')
        if DarkCentre:
            ax.scatter(0,0,s=psz,color='slategrey')
        ax.scatter(nopincoord[0],nopincoord[1],s=psz*(pelletDiam/pinDiam)**2,color='white')
        ax.set_aspect('equal')
        
        if mode=='plot': 
            plt.show()
        else:
            return ax
        
        
        
        
# -- Assembly testing function --

def assembly_test():
    
    print('\033[1m_test():\033[0m Assemblies are declared through a .txt file containing all spectra:')
    print('  -->  x=assembly(\'Assembly17TestData.txt\')')
    print('\033[1m_test():\033[0m We can remove pins individually and plot the assembly arrangement.')
    print('  -->  x.pop_pin([[0,0],[-3,5],[-5,-6],[1,2],[2,3]])')
    print('  -->  x.draw()')
    x=assembly('Assembly17TestData.txt')
    x.pop_pin([[0,0],[-3,5],[-5,-6],[1,2],[2,3]])
    x.draw()
    
    print('\033[1m_test():\033[0m Some other things we can do with pins are:')
    print('  -->  x.pop_pin([\'All\')]')
    print('  -->  x.add_pin([[1,1],[-3,5]],[\'4JR_atm106_spectrum.txt\',\'4JR_moxrod_spectrum2.txt\'])')
    print('  -->  x.add_pin([[.0250,.0375]],[\'4JR_tmirod_uo2_spectrum.txt\'],by=\'coordinates\')') 
    print('  -->  x.add_pin([[-2,-1]],[\'convex\'],by=\'index\')') 
    x.pop_pin(['All'])
    x.add_pin([[1,1],[-3,5]],['4JR_atm106_spectrum.txt','4JR_moxrod_spectrum2.txt'])
    x.add_pin([[.0250,.0375]],['4JR_tmirod_uo2_spectrum.txt'],by='coordinates')
        
    print('\033[1m_test():\033[0m Assemblies can also be plotted within an outside figure.')
    print('  -->  fig,ax=plt.subplots(figsize=(3,3))')
    print('  -->  ax=x.draw(ax,size=3)')
    print('  -->  plt.show()')
    fig,ax=plt.subplots(figsize=(3,3))
    ax=x.draw(ax,size=3)
    plt.show()
    
    return x
        
    

In [8]:
            # -- TESTING BLOCK --
TEST=0
if TEST:
    
    x=assembly_test()

<h2>The Detector Class</h2>

Once we have definded the assembly class we are going to define the detector class. This class will have the following attributes:

- The assembly which the detector measures (```self.assembly```).
- The detector's position with respect to the central pin of the assembly (```self.placement```). The variable will be stored in the corresponding polar coordinates. We do not call the atribute position to avoid confusion with the position system for assemblies.
- The currently measured spectre by the detector of the assembly (```self.spectre```). This will be computed by a class method called ```self.compute_spectre()```.

Said calculation of the spectre $\Phi$ will be the sum of the individial contributions of the present pins, and those will be the individual spectra $\phi_k$ corrected by a $D_k^2/d_{k}^2$ factor, where $D_k$ is the distance to the detector when the spectra were taken and $d_{k}$ is the distance from the pin to the current detector. That gives us, 
$$\Phi=\sum_k\frac{D_k^2}{d_k^2}\phi_k\ .$$
Also, we can see in the drawn spectra above (in the testing block) that the range of energies for which the meaures are taken vary. For the combined spectre we will take the smallest range. The binning intervals and origin are the same so we can sum fluence in channels easily.

In [9]:
class detector:
    
    def __init__(self,assemb,place=(2,0),coordinates='polar',compSpectra=True):
        
            # We assign the detector's assembly
        if type(assemb)!=assembly:
            raise Exception('Missing assembly instance!')
        self.assembly=assemb
        
            # We assign the detector's location
        if np.shape(place)!=(2,):
            raise ValueError('Position must be a two-tuple.')
        
        if coordinates=='polar':
            if place[0]<0:
                raise ValueError('Radius in polar coordinates must be non-negative!')
        elif coordinates=='cartesian':
            place=[np.linalg.norm(place),np.arctan2(place[1],place[0])]
        else:
            raise Exception('Unavailable coordinate system. Must be \'polar\' or \'cartesian\'.')
        
        self.placement=np.array(place)
        
        minRadius=np.floor(self.assembly._nPinSide/2)*self.assembly.gap*np.sqrt(2)+pinDiam/2
        if self.placement[0]<=minRadius:
            print('WARNING: Detector rotates through assembly.')
            print('Minimal radius ',minRadius)
        
            # We assign the detector's base and current spectre
        if compSpectra:
            self.spectrebase=self._compute_spectrebase()
            self.spectre=self._compute_spectre()
        else:
            self.spectrebase=None
            self.spectre=None
    
    def _pinDistance(self,PinList,by='position'):
        '''To compute the distance between a set of pins and the current position of the detector.
        
        pinDistance(self,PinList,by='position')
        
        The variable PinList must be a list of two-tuples. Each tuple will represent a pin to to calculate the 
        distance to the detector. The second argument specifies the location system in which the two-tuples are 
        given.
        '''
        
        # We make sure the input has the correct shape
        PinList=np.array(PinList)
        if np.shape(PinList)[1]!=2: 
            raise ValueError('Must provide a list of two-tuples')
        
        # We write the two-tuples in the coordinate system
        if by=='index':  
            PinList=self.assembly.idx2pos(PinList) 
            by='position' 
        
        if by=='coordinates':   
            # We check if all entries represent a multiple of self.assembly.gap 
            AddPinList=AddPinList/self.assembly.gap 
            by='position'
            e=1E-6 
            if not np.all(PinList-np.round(PinList)<e*np.ones(np.shape(PinList))):   
                raise Exception('Coordinates do not coincide with pin location')
            
        if by=='position':
            # We check if the tuple is consistent with the position system
            centre=(self.assembly._nPinSide-1)/2
            if np.max(abs(PinList))>centre: 
                raise Exception('Positions out of range.')
                
            # We finally compute the distance
            PinList=PinList*self.assembly.gap 
            by='coordinates' 
            return np.array([np.linalg.norm(self.get_coordinates()-PinList[i]) for i in range(len(PinList))])
            
        else:
            raise Exception('List is not given in any available location system.')
            
    
    # -- Convenience Methods --
            
    def rotate(self,deltaAngle=np.pi/2):
        '''To rotate the detector around the central pin an angle deltaAngle.
        
        rotate(self,deltaAngle=np.pi/2)
        
        The angle must be given in radians.
        '''
        
        self.placement[1]=self.placement[1]+deltaAngle
        self.update_spectre()
        
    def set_angle(self,angle):
        self.placement[1]=angle
        self.update_spectre()
        
    def set_radius(self,dist):
        self.placement[0]=dist
        self.update_spectre()
        
    def set_coordinates(self,place):
        self.placement=[np.linalg.norm(place),np.arctan2(place[1],place[0])]
        self.update_spectre()
        
    def get_coordinates(self):
        return self.placement[0]*np.array([np.cos(self.placement[1]),np.sin(self.placement[1])])
    
    
    # -- Spectre computation methods --        
    
    def _compute_spectre(self):
        '''To compute the spectre read by the detector as a sum of the contributions of all pins corrected by 
        a factor inversely proportional to the square of the distance.
        
        _compute_spectre(self)
        
        Returns a spectre instance (if there are no pins returns 0).
        '''
        
        s=self.assembly._nPinSide   # We simply rename it
        
        FirstSpectreRead=0
        j=0
        while (not FirstSpectreRead) and j<s**2:
            row=int(j/s)
            col=j%s
            if self.assembly.spectra[row][col]!='-':
                if self.assembly.spectra[row][col] in SpectraDict:
                    spect=copy.deepcopy(SpectraDict[self.assembly.spectra[row][col]])
                elif self.assembly.spectra[row][col]=='convex': 
                    spect=copy.deepcopy(SpectraDict[list(SpectraDict.keys())[0]])
                    spect.data[1]=self.assembly.pinslots[row][col][0]*spect.data[1]
                    for i in range(1,len(SpectraDict)):
                        #####
                        nextSpect=copy.deepcopy(SpectraDict[list(SpectraDict.keys())[i]])
                        lenDif=len(nextSpect.data[0])-len(spect.data[0])
                        if lenDif>0:
                            nextSpect.data=np.delete(nextSpect.data,np.arange(-lenDif,0),axis=1) 
                        elif lenDif<0:
                            spect.data=np.delete(spect.data,np.arange(lenDif,0),axis=1)
                        spect.data[1]=spect.data[1]+self.assembly.pinslots[row][col][i]*nextSpect.data[1]
                        #####
                        #spect.sum_spectre(SpectraDict[list(SpectraDict.keys())[i]],self.assembly.pinslots[row][col][i])
                else: 
                    spect=spectre(self.assembly.spectra[row][col])
                distCorrFactor=(spect.distance/self._pinDistance([[row,col]],by='index')[0])**2
                spect.data[1]=spect.data[1]*distCorrFactor
                ####
                #spect.data[2]=(spect.data[1]*spect.data[2])**2
                ####
                        # We work with variance instead of relative error for ease of addition.
                FirstSpectreRead=1
            j=j+1
        if FirstSpectreRead==0:
            print('No spectre read.')
            return 0
    
        for i in range(j,s**2):
            row=int(i/s)
            col=i%s
            if self.assembly.spectra[row][col]!='-':
                if self.assembly.spectra[row][col] in SpectraDict:
                    newSpect=copy.deepcopy(SpectraDict[self.assembly.spectra[row][col]])
                elif self.assembly.spectra[row][col]=='convex':  
                    newSpect=copy.deepcopy(SpectraDict[list(SpectraDict.keys())[0]])
                    newSpect.data[1]=self.assembly.pinslots[row][col][0]*newSpect.data[1]
                    for i in range(1,len(SpectraDict)):
                        #####
                        nextSpect=copy.deepcopy(SpectraDict[list(SpectraDict.keys())[i]])
                        lenDif=len(nextSpect.data[0])-len(newSpect.data[0])
                        if lenDif>0:
                            nextSpect.data=np.delete(nextSpect.data,np.arange(-lenDif,0),axis=1) 
                        elif lenDif<0:
                            newSpect.data=np.delete(newSpect.data,np.arange(lenDif,0),axis=1)
                        newSpect.data[1]=newSpect.data[1]+self.assembly.pinslots[row][col][i]*nextSpect.data[1]
                        #####
                        #newSpect.sum_spectre(SpectraDict[list(SpectraDict.keys())[i]],self.assembly.pinslots[row][col][i])
                else: 
                    newSpect=spectre(self.assembly.spectra[row][col])
                
                #####
                lenDif=len(newSpect.data[0])-len(spect.data[0])
                if lenDif>0:
                    newSpect.data=np.delete(newSpect.data,np.arange(-lenDif,0),axis=1) 
                elif lenDif<0:
                    spect.data=np.delete(spect.data,np.arange(lenDif,0),axis=1)
                #####
                
                    # We do the proceeding operations. 
                distCorrFactor=(newSpect.distance/self._pinDistance([[row,col]],by='index')[0])**2
                #spect.sum_spectre(newSpect,distCorrFactor)    #####
                
                #####
                spect.data[1]=spect.data[1]+distCorrFactor*newSpect.data[1]
                #newSpect.data[2]=(distCorrFactor*newSpect.data[1]*newSpect.data[2])**2
                    # We add the variance
                #spect.data[2]=spect.data[2]+newSpect.data[2]
        
        #no0=np.where(spect.data[1]!=0)[0]
        #spect.data[2]=[spect.data[2][i]/spect.data[1][i] if i in no0 else 0 for i in range(len(spect.data[2]))]
                    # We return to relative error
                #####
        
            # We update the spectre name
        spect.name='detector at (%.2f, %.2f), $\\theta=$%.2f.'%(self.get_coordinates()[0],
                                                                self.get_coordinates()[1],
                                                                self.placement[1])
        spect.distance=self.placement[0]
                
        return spect
    
    def _compute_spectrebase(self):
        ''' To compute the spectre at angle 0 as a reference.
        '''
            
        angle=self.placement[1]
        self.placement[1]=0
        spectrebase=self._compute_spectre()
        self.placement[1]=angle
        if spectrebase==0:
            return 0
        
            # We update the spectre name
        spectrebase.name='detector spectrebase'

        return spectrebase
    
    def update_spectre(self,):
        if self.spectre!=None: self.spectre=self._compute_spectre()
        
    def update_spectrebase(self):
        if self.spectrebase!=None: self.spectrebase=self._compute_spectrebase()
    
    def update(self):
        self.update_spectre()
        self.update_spectrebase()
        
        
    # -- Plotting methods --
        
    def draw_geometry(self,ax=None,size=4.82,pinsize='default',DarkCentre=1):
        '''A visual plot of the detector with the assembly.
        
        plot(self,figsize=4.82,pinsize='default',DarkCentre=1)
        
        DarkCentre is a variable to control whether we want to colour the central pin in a greyish colour so 
        it is more easily picked out.
        '''
            
        # We generate the circle
        theta=np.linspace(0,2*np.pi,150)
        X=self.placement[0]*np.array([np.cos(theta),np.sin(theta)])
        
        mode='axes'
        if ax==None:
            fig,ax=plt.subplots(figsize=(size,size))
            mode='plot'
        
        assemblyPlotSize=size*(np.floor(self.assembly._nPinSide/2)*self.assembly.gap/self.placement[0])
            # pin Size might be wrong if the detector radius is smaller than minRadius
        
        ax=self.assembly.draw(ax,size=assemblyPlotSize)
        
        ax.plot(X[0],X[1],color='red',linewidth=0.5)
        markerAngle=(self.placement[1]+np.pi/4)*(180/np.pi)
        detCoord=self.get_coordinates()
        psd=((pinDiam/self.assembly.gap)*(assemblyPlotSize/(self.assembly._nPinSide-1))*35.4*1.3)**2
        ax.scatter(detCoord[0],detCoord[1],marker=(4,0,markerAngle),s=psd,color='red')
        
        if mode=='plot': 
            plt.show()
        else:
            return ax
        
    def draw_here(self,fig=None,figsize=(9, 4),Eunits='keV',yError=0,linthresh=None):
        '''A visual display of all the information to do with the detector in its current placement.
        
        draw_here(self,fig=None,figsize=(9, 4),Eunits='keV',yError=0,linthresh=None)
        
        '''

        fig = plt.figure(figsize=figsize,layout='compressed')
        
        wSep=0.1
        ratio=[6.5, 10]
        subfigs = fig.subfigures(1, 2, wspace=wSep, width_ratios=ratio)
        
        assemblySize=(ratio[0]/np.sum(ratio))*(figsize[0]-wSep)
        print(assemblySize)
        axLeft=subfigs[0].subplots()
        axLeft=self.draw_geometry(axLeft,assemblySize) # We have to pass assemblySize for the dot size.
        
        #spectraLenght=(ratio[1]/np.sum(ratio))*(figsize[0]-wSep)
        axRight=subfigs[1].subplots(2,sharex=True)
        axRight[0]=self.spectre.draw(axRight[0],Eunits=Eunits,yError=yError)
        if self.placement[1]==0:
            print('At angle 0 the spectre comparison is 0.')
        else:
            axRight[1]=self.spectre.compare(self.spectrebase,axRight[1],(0,0),Eunits,yError,linthresh=linthresh)
        plt.show()
        
        print('WARNING: This is an overall outlook and detail may be lost in the spectre plots.')
        print('For a more accurate representation, plot separately with figure size as big as possible.')
        
        
    # -- Assembly modification methods --
        # Methods to modify the detector's assembly instance directly so that the spectres are accordingly 
        # updated
        
    def pop_pin(self,PopPinList,by='position'):
        '''To remove pins from self.assembly.pinslots and afterwards update the spectres.
        
        pop_pin(self,PopPinList,by='position')
        
        '''
        
        self.assembly.pop_pin(PopPinList,by)
        self.update()
    
    def add_pin(self,AddPinList,SpectraList,by='position'):
        '''To add pins to self.assembly.pinslots and afterwards update the spectres.
        
        add_pin(self,AddPinList,SpectraList,by='position')
        
        '''
        
        self.assembly.add_pin(AddPinList,SpectraList,by)
        self.update()
        
    def pop_pinRand(self,nPins=1):
        '''To remove nPins randombly from the array.
        '''
        
        poppedPins=self.assembly.pop_pinRand(nPins)
        self.update()
        
        return poppedPins
 

    # -- Generation and representation of data for neural network analysis -- 
    
    def gen_dataentry(self,nAngles=32,include2pi=False,startAngle=0):

        currentAngle=self.placement[1]
        
        self.set_angle(startAngle)
        fluences=self._compute_spectre().data[:-1] # We take the energies and the fluence 
        
        angles=np.linspace(startAngle,startAngle+2*np.pi,nAngles+1)
        angles=np.delete(angles,0)
        if not include2pi:
            angles=np.delete(angles,-1)
            
        for angle in angles:
                #self.set_angle(angle) without the variances
            self.placement[1]=angle
            fluences=np.concatenate((fluences,self._compute_spectre().data[1][np.newaxis]),axis=0)
        
        #fluences=np.delete(fluences,0,axis=1) # Removal of the fluence 0 entry
        
        #energies=fluences[0]
        #fluences=np.delete(fluences,0,axis=0) # Removal of the energies vector
        self.set_angle(currentAngle)
        
        return fluences
    
    def draw_dataentry(self,nAngles=32,include2pi=False,startAngle=0,Erange=(None,None),Eunits='keV',
                       EGap=(50,0.050),Frange=(None,None),figsize=(11,5)):
        draw_dataentry(self.gen_dataentry(nAngles,include2pi,startAngle),
                       Erange,Eunits,EGap,Frange,figsize=figsize)
        
    def draw(self,nAngles=32,include2pi=False,startAngle=0,Erange=(None,None),Eunits='keV',
                       EGap=(50,0.050),Frange=(None,None),figsize=(11,3.5),warning='on',savefig='-'):
        '''A visual display of the detector and the dataentry it generates.
        
        draw(self,nAngles=32,include2pi=False,startAngle=0,Erange=(None,None),Eunits='keV',
                       EGap=(50,0.050),Frange=(None,None),figsize=(11,3.5),warning='on',savefig='-')
                       
        savefig is the string name for the saved figure.
        '''
            
        EFluence=self.gen_dataentry(nAngles,include2pi,startAngle)

        ratio=[6.5,10]
        fig,ax=plt.subplots(1,2,figsize=figsize,width_ratios=ratio)
        
        wSep=0.1
        assemblySize=(ratio[0]/np.sum(ratio))*(figsize[0]-2-wSep)
        ax[0]=self.draw_geometry(ax[0],assemblySize)# We have to pass assemblySize for the dot size.
        img,cmap,ax[1]=draw_dataentry(EFluence,Erange,Eunits,EGap,Frange,ax[1])
        plt.colorbar(img,cmap=cmap) 
        if savefig!='-':
            plt.savefig(savefig,bbox_inches='tight') ###
        plt.show()
        
        if warning=='on':
            print('WARNING: This is an overall outlook and detail may be lost in the spectre plots.')
            print('For a more accurate representation, plot separately with figure size as big as possible.')
        
    
      
# -- Functions defined outside the detector class --
        
def draw_dataentry(EFluences,Erange=(None,None),Eunits='keV',EGap=(50,0.050),Frange=(None,None),
                   ax=None,figsize=(11,5)):
        
    mode='axes' 
    if ax==None:
        fig, ax = plt.subplots(figsize=figsize)
        mode='plot'
    
        # - Energy axis -  
    Energies=EFluences[0]
    fluences=np.delete(EFluences,0,axis=0)
        
    if Eunits=='keV':
        ax.set_xlabel('Energy (keV)')
        energies=1000*Energies
        if not isinstance(EGap,(int,float)): EGap=EGap[0]
    elif Eunits=='MeV':
        ax.set_xlabel('Energy (MeV)')
        energies=Energies
        if not isinstance(EGap,(int,float)): EGap=EGap[1]
    else:
        raise Exception('Wrong selection of energy units.')
    
    # We generate a two-vector-array with the Energies and their indices
    Eax=np.concatenate((np.arange(len(energies))[np.newaxis],energies[np.newaxis]),axis=0)
        
    # We establish the limiting Energy indices
    i=0
    if Erange[0]==None: 
        EMinIdx=0
    else:
        while energies[i]<Erange[0] and i<len(energies)-1:
            i+=1
        EMinIdx=i
    if Erange[1]==None: 
        EMaxIdx=len(energies)-1
    else:
        while energies[i]<Erange[1] and i<len(energies)-1:
            i+=1
        EMaxIdx=i
       
    # We take the energies multiples of the enerGap
    Eax=Eax[:,[energies.tolist().index(energy) for energy in energies[EMinIdx:EMaxIdx+1] 
               if energy%EGap==0]].T
    # We add the first energy value
    Eax=np.append([[0,np.min(energies)]],Eax,axis=0).T
    Eax[0]=[idx-EMinIdx for idx in Eax[0]]
    ax.set_xticks(ticks=Eax[0],labels=Eax[1])
    
        # - Colour plot - 
    cmapFluence = matplotlib.colors.LinearSegmentedColormap.from_list(
            'fluence_colormap',['green','yellow','red'],256)
    
    data2plot=fluences[:,EMinIdx:EMaxIdx+1]
    img = ax.imshow(data2plot, cmap=cmapFluence, norm='log', aspect='auto', interpolation='nearest', 
                    vmin=Frange[0], vmax=Frange[1], origin='lower')
    
        # - Angle axis -
    Aax=[[],[]]
    Aaxminor=[[],[]]
    nAngles=np.shape(data2plot)[0]
    if nAngles%2==0:
        Aax=np.append(Aax,[[0,nAngles/2],['0','$\pi$']],axis=1)
    if nAngles%4==0:
        Aax=np.append(Aax,[[nAngles/4,3*nAngles/4],['$\pi$/4','3$\pi$/4']],axis=1)
    if nAngles%8==0 and figsize[1]>=5:
        Aaxminor=np.append(Aaxminor,[[nAngles/8,3*nAngles/8,5*nAngles/8,7*nAngles/8],
                                     ['$\pi$/8','3$\pi$/8','5$\pi$/8','7$\pi$/8']],axis=1)
    if nAngles%16==0 and figsize[1]>=10:
        Aaxminor=np.append(Aaxminor,[[nAngles/16,3*nAngles/16,5*nAngles/16,7*nAngles/16,
                                      9*nAngles/16,11*nAngles/16,13*nAngles/16,15*nAngles/16],
                                     ['$\pi$/16','3$\pi$/16','5$\pi$/16','7$\pi$/16',
                                      '9$\pi$/16','11$\pi$/16','13$\pi$/16','15$\pi$/16']],axis=1)
    ax.set_yticks(ticks=[float(num) for num in Aax[0]],labels=Aax[1])
    ax.set_yticks(ticks=[float(num) for num in Aaxminor[0]],labels=Aaxminor[1],minor=True)
    ax.tick_params(which='major', length=4)
    ax.tick_params(which='minor', length=2)
    
    ax.set_title('Fluence ($\gamma$/cm$^2$)')
    ax.set_ylabel('Detector angle $\\theta$ (rads)')
    
    if mode=='plot': 
        plt.colorbar(img,cmap=cmapFluence)
        plt.show()
        if figsize[0]<=6:
            print('WARNING: Detail may be lost in small figure sizes.')
            print('For the most accurate representation, increase figure size as much as possible.')
    else:
        return (img,cmapFluence,ax)
        
        
# -- Detector testing function -- 

def detector_test(x):
    
    print('\n\033[1m_test():\033[0m Detectors are given an assembly and placement in polar coordinates when \
declared.')
    print('  -->  det=detector(x,(0.18,np.pi/4))')
    det=detector(x,(0.18,np.pi/4))
    print('  -->  det.placement')
    print(det.placement)
    print('\033[1m_test():\033[0m To rotate detectors and get their cartesian coordinates:')
    print('  -->  det.rotate(-3*np.pi/4)')
    det.rotate(-3*np.pi/4)
    print('  -->  det.get_coordinates()')
    print(det.get_coordinates())
    print('\033[1m_test():\033[0m Detector\'s geometry can be drawn.')
    print('  -->  det.rotate(-np.pi/6)')
    print('  -->  det.draw_geometry()')
    det.rotate(-np.pi/6)
    det.draw_geometry()
    
    print('\n\033[1m_test():\033[0m When detectors are declared a spectre instance is computed with the \
contributions of all the pins in the assembly. This is accessed through det.spectre .')
    print('\033[1m_test():\033[0m A second spectre is computed as reference at $\\theta=0$. It also is a \
detector atribute: det.spectrebase .')
    
    print('\n\033[1m_test():\033[0m We can manually set interesting angles for the detector.')
    print('  -->  det.set_angle(3.0785026841998357)')
    det.set_angle(3.0785026841998357)   # An interesting angle
    print('\033[1m_test():\033[0m and plot their geometry together with their spectra.')
    print('  -->  det.draw_here()')
    det.draw_here()
    
    print('\n\033[1m_test():\033[0m Pins can be added directly to the detector\'s assembly.')
    print('  -->  det.add_pin([[-2,-3]],[\'4JR_tmirod_uo2_spectrum.txt\'])')
    print('  -->  det.draw()')
    det.add_pin([[-2,-3]],['4JR_tmirod_uo2_spectrum.txt'])
    det.draw_here()
    print('\n\033[1m_test():\033[0m det.assembly.add_pin() should be avoided to save trouble with \
det.spectre .')
    
    print('\n\033[1m_test():\033[0m Detectors will warn of collision with assembly.')
    print('  -->  x2=assembly()')
    print('  -->  x2.add_pin([[8,8]],[\'4JR_tmirod_uo2_spectrum.txt\'])')
    print('  -->  det2=detector(x2,(0.13,np.pi))')
    print('  -->  det.draw_here()')
    x2=assembly()
    x2.add_pin([[8,8]],['4JR_tmirod_uo2_spectrum.txt'])
    det2=detector(x2,(0.13,np.pi))
    det2.draw_here()
    
    return (det,det2)

def dataentry_test(det,det2):
    
    print('\n\033[1m_test():\033[0m Let us begin working with dataentries. \
We begin by generating two dataentries of the previous detectors.')
    print('  -->  EFluences2=det2.gen_dataentry(64)')
    print('  -->  EFluences=det.gen_dataentry(128)')
    EFluences2=det2.gen_dataentry(64)
    EFluences=det.gen_dataentry(128)
    
    print('\n\033[1m_test():\033[0m We can visualise generated dataentries using')
    print('  -->  draw_dataentry(EFluences2)')
    draw_dataentry(EFluences2)
    
    print('\n\033[1m_test():\033[0m As before, alternatively we can plot within the code.')
    print('  -->  fig,ax=plt.subplots(1,2,figsize=(10,4))')
    print('  -->  img0,cmap0,ax[0]=draw_dataentry(EFluences2,EGap=100,ax=ax[0])')
    print('  -->  img1,cmap1,ax[1]=draw_dataentry(EFluences,EGap=100,Frange=(None,3e-15),ax=ax[1])')
    print('  -->  plt.colorbar(img0,cmap=cmap0)')
    print('  -->  ax[1].set_ylabel(\'\')')
    print('  -->  plt.colorbar(img1,cmap=cmap1)')
    print('  -->  plt.show()')
    fig,ax=plt.subplots(1,2,figsize=(10,4))
    img0,cmap0,ax[0]=draw_dataentry(EFluences2,EGap=100,ax=ax[0])
    img1,cmap1,ax[1]=draw_dataentry(EFluences,EGap=100,Frange=(None,3e-15),ax=ax[1])
    plt.colorbar(img0,cmap=cmap0)
    ax[1].set_ylabel('')
    plt.colorbar(img1,cmap=cmap1)
    plt.show()
    print('\033[1m_test():\033[0m Again, note the effect of plot size on the accuracy of the representation.')
    
    print('\n\033[1m_test():\033[0m Finally, dataentries can be generated and plotted directly from the \
detector instance with one command.')
    print('  -->  det2.draw_dataentry(512)')
    det2.draw_dataentry(512)
    
            

In [10]:
            # -- TESTING BLOCK --
# TEST=0
if TEST:
    
    det,det2=detector_test(x)

In [11]:
            # -- TESTING BLOCK --
# TEST=0
if TEST:
    
    dataentry_test()

<h2>Global functions</h2>

In [12]:
def NA_test():
    spectre_test()
    x=assembly_test()
    det,det2=detector_test(x)
    dataentry_test(det,det2)
    return det,det2